In [2]:
import gurobipy as gp
from gurobipy import GRB
## 

In [3]:
# !pip install gurobipy


In [4]:
from gurobipy import *

### Step 1: compute minimum tardy amount t

##### Constants
$\text{Let}$  <br>

$M = \{1, 2, ..., 5\} \text{ be the set of machines,}$<br>

$J = \{1, 2, ..., 12\} \text{ be the set of jobs,}$  <br>

$I = \{1, 2, ..., 6\}\text{ be the set of process number,}$ <br>

$S_{j} = \text{ the splitting timing for job } j \text{, } \forall j \in J \text{, }$ <br>

$D_{j} = \text{ the due time for job } j \text{, } \forall j \in J \text{, }$ <br>

$P = \{boiling, baking, smoking, null\} \text{ be the set of process name,}$ <br>

$ C_{mp} = 
\begin{cases}
    \ 1, & \text{if machine } m \text{ can do process } p \\
    0,     & \text{otherwise}
\end{cases}
\text{, }\forall m \in M, p \in P
$

$A_{ijp} = 
\begin{cases}
    \ 1, & \text{if process } i \text{ in job } j \text{ is } p \\
    0,     & \text{otherwise}
\end{cases}
\text{, }\forall i \in I, j \in J, p \in P\text{, }$

$T_{ij} = \text{time spent by process } i \text{ of job } j \text{ (} T_{ij} = 0 \text{ if } \sum_{p \in P}{A_{ijp}} = 0 \text{) } \forall i \in I, j \in J \text{. }$

$ b= \text{the opening time of the smoking station}$


##### Variables
$\text{def.}$


$ x_{ijm} = 
\begin{cases}
    \ 1, & \text{if the process } i \text{ of job }j \text{ is assign to machine }m\\
     0,     & \text{otherwise}
\end{cases}
\forall i \in I, j \in J
$

$w_{j} = 
\begin{cases}
    \ 1, & \text{if job } j \text{ is tardy job }\\
     0,     & \text{otherwise}
\end{cases}
\forall i \in I
$

<!-- $ x_{ijm} = 
\begin{cases}
    \ 1, & \text{if the process } i \text{ of job }j \text{ is assign to machine }m\\
     0,     & \text{otherwise}
\end{cases}
\forall i \in I, j \in J
$ -->

$z_{i,j,i',j'} = 
\begin{cases}
    \ 1, & \text{if job } j \text{ of process }i \text{ is scheduled before job }j' \text{ of process } i'\\
     0,     & \text{otherwise}
\end{cases}
\forall i,i' \in I, j,j' \in J
$

$ y_{i,j,i',j',m} =
\begin{cases}
    \ 1, & \text{if } x_{i,j,m} = x_{i',j',m} = 1 \\
     0,     & \text{otherwise}
\end{cases}
\forall i,i' \in I, j,j' \in J, m \in M
$

$ v_{ij} = \text{the completion time of process }i \text{, job }j, \forall i \in I, j \in J
$







$\text{obj. }$ $\min \sum_{j \in J} w_j$ 

$\text{s.t. }$  \
ensure the correctness of order of each process 

**1-a. same job** \
$v_{ij} + T_{ij} - v_{i'j} \leq K \cdot z_{iji'j} \quad \forall i, i' \in I,\: j, j' \in J,\: i' < i,\: j' < j$ \
$v_{i'j} + T_{i'j} - v_{ij} \leq K \cdot (1-z_{iji'j}) \quad \forall i, i' \in I,\: j, j' \in J,\: i' < i,\: j' < j$   

**1-b. same machine** \
$y_{iji'j'm}(v_{ij} \cdot x_{ijm} + T_{ij} - v_{i'j'} \cdot x_{i'j'm}) \leq K \cdot z_{iji'j'} \quad \forall i, i' \in I,\: j, j' \in J,\: m \in M, \: i' < i,\: j' < j$ \
$y_{iji'j'm}(v_{i'j'} \cdot x_{i'j'm} + T_{i'j'} - v_{ij} \cdot x_{ijm}) \leq K \cdot (1 - z_{iji'j'}) \quad \forall i, i' \in I,\: j, j' \in J,\: m \in M, \: i' < i,\: j' < j$

**2. Each process is completed by exactly one machine**

$$\sum_{m \in M}x_{ijm} = 1, \forall i \in I, j \in J$$

**3. Whether a machine can do to process type** \
$x_{ijm} \cdot A_{ijp} \leq C_{mp} \quad \forall p \in P, i \in I, j \in J, m \in M$

**4. Constraint related to "Split"**

$$v_{i+1j}-v_{ij}=T_{i+1j}, \forall i \in I - \{S_{j}, |I|\}, j \in J$$ :      連續

$$x_{i+1jm} = x_{ijm}, \forall i \in I - \{S_{j}, |I|\}, m \in M, j \in J$$:    同一台機器

**5. Minimum Tardiness**\
$v_{|I|j} - (D_j - b) \geq K \cdot w_j \quad \forall j \in J$ 

**6. Non-negative constraints** \
$w_{ij} \in \{0,1\}$


### Step 2: minimize makespan

$\text{obj. } \min \: (\max v_{ij})$ \
$\text{s.t.}$ \
$\sum_{j \in J}w_j = t$  
$\text{and all above constraints.}$ 

In [5]:
import pandas as pd

In [6]:
# data = pd.read_excel('openpyxl","data/OR110-1_case01.xlsx', engine='openpyxl')
df1 = pd.read_excel ('data/OR110-1_case01.xls', sheet_name='Instance 1')
df2 = pd.read_excel ('data/OR110-1_case01.xls', sheet_name='Instance 2')
df3 = pd.read_excel ('data/OR110-1_case01.xls', sheet_name='Instance 3')



In [7]:
df1

,Job ID,Process 1,Process 2,Splitting Timing,Process 1.1,Process 2.1,Due Time
0,1,Boiling,Baking,1,2.7,1.3,12:30:00
1,2,Baking,Boiling,1,1.6,1.4,12:30:00
2,3,Boiling,Baking,1,0.7,1.9,12:30:00
3,4,Smoking,Boiling,1,0.5,0.7,12:30:00
4,5,Boiling,Baking,1,0.8,1.0,12:30:00
5,6,Boiling,NaN,0,2.5,NaN,12:30:00
6,7,Baking,Smoking,1,1.4,2.0,12:30:00
7,8,Baking,Smoking,1,1.1,1.1,17:30:00
8,9,Boiling,Baking,1,0.8,0.7,17:30:00
9,10,Baking,Boiling,1,1.0,0.5,17:30:00


# Problem 1

In [8]:
import numpy as np

In [9]:
K = 1000000

In [10]:
from datetime import datetime, date

date_time_str = "7:30:00"

b = datetime.strptime(date_time_str, '%H:%M:%S').time()


print ("The date is", b)

The date is 07:30:00


In [11]:
S = []
D = []
for index, row in df1.iterrows():
    S.append(row['Splitting Timing'])
    D.append(row['Due Time'])
D

[datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(17, 30),
 datetime.time(17, 30),
 datetime.time(17, 30),
 datetime.time(17, 30),
 datetime.time(17, 30)]

In [35]:
for i in range(len(S)):
    S[i] = 0
S

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
peg1 = Model("peg1")

M = 4
I = 2
J = 12
method = ["Boiling", "Baking", "Smoking"]
method.append("null")
P = len(method)

A = {}
for i in range(I):
    # A.append([])
    for j in range(J):
        # A[i].append([])
        count = 0
        for p in range(P-1):
            if(df1["Process "+ str(i+1)][j] == method[p]):
                A[i,j,p] = 1
                count = 1
            else:
                A[i,j,p] = 0
        if(count == 0):
            A[i, j, P-1] = 1
        else:
            A[i, j, P-1] = 0
A

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-31


{(0, 0, 0): 1,
 (0, 0, 1): 0,
 (0, 0, 2): 0,
 (0, 0, 3): 0,
 (0, 1, 0): 0,
 (0, 1, 1): 1,
 (0, 1, 2): 0,
 (0, 1, 3): 0,
 (0, 2, 0): 1,
 (0, 2, 1): 0,
 (0, 2, 2): 0,
 (0, 2, 3): 0,
 (0, 3, 0): 0,
 (0, 3, 1): 0,
 (0, 3, 2): 1,
 (0, 3, 3): 0,
 (0, 4, 0): 1,
 (0, 4, 1): 0,
 (0, 4, 2): 0,
 (0, 4, 3): 0,
 (0, 5, 0): 1,
 (0, 5, 1): 0,
 (0, 5, 2): 0,
 (0, 5, 3): 0,
 (0, 6, 0): 0,
 (0, 6, 1): 1,
 (0, 6, 2): 0,
 (0, 6, 3): 0,
 (0, 7, 0): 0,
 (0, 7, 1): 1,
 (0, 7, 2): 0,
 (0, 7, 3): 0,
 (0, 8, 0): 1,
 (0, 8, 1): 0,
 (0, 8, 2): 0,
 (0, 8, 3): 0,
 (0, 9, 0): 0,
 (0, 9, 1): 1,
 (0, 9, 2): 0,
 (0, 9, 3): 0,
 (0, 10, 0): 1,
 (0, 10, 1): 0,
 (0, 10, 2): 0,
 (0, 10, 3): 0,
 (0, 11, 0): 0,
 (0, 11, 1): 0,
 (0, 11, 2): 1,
 (0, 11, 3): 0,
 (1, 0, 0): 0,
 (1, 0, 1): 1,
 (1, 0, 2): 0,
 (1, 0, 3): 0,
 (1, 1, 0): 1,
 (1, 1, 1): 0,
 (1, 1, 2): 0,
 (1, 1, 3): 0,
 (1, 2, 0): 0,
 (1, 2, 1): 1,
 (1, 2, 2): 0,
 (1, 2, 3): 0,
 (1, 3, 0): 1,
 (1, 3, 1): 0,
 (1, 3, 2): 0,
 (1, 3, 3): 0,
 (1, 4, 0): 0,
 (1, 4, 1): 1,
 (

In [13]:
T = {}
for i in range(I):
    # temp = []
    for j in range(J):
        if(np.isnan(df1["Process " + str(i+1) + ".1"][j])):
            # temp.append(0)
            T[i, j] = 0
        else:
            # temp.append(j)
            T[i, j] = df1["Process " + str(i+1) + ".1"][j]
    # T.append(temp)
T

{(0, 0): 2.7,
 (0, 1): 1.6,
 (0, 2): 0.7,
 (0, 3): 0.5,
 (0, 4): 0.8,
 (0, 5): 2.5,
 (0, 6): 1.4,
 (0, 7): 1.1,
 (0, 8): 0.8,
 (0, 9): 1.0,
 (0, 10): 3.0,
 (0, 11): 2.0,
 (1, 0): 1.3,
 (1, 1): 1.4,
 (1, 2): 1.9,
 (1, 3): 0.7,
 (1, 4): 1.0,
 (1, 5): 0,
 (1, 6): 2.0,
 (1, 7): 1.1,
 (1, 8): 0.7,
 (1, 9): 0.5,
 (1, 10): 1.4,
 (1, 11): 0}

In [14]:

C = {}
machineNumber = 4
for i in range(machineNumber):
    for j in range(P):
        C[i, j] = 1
#     C[i, P-1] = 0
C


{(0, 0): 1,
 (0, 1): 1,
 (0, 2): 1,
 (0, 3): 1,
 (1, 0): 1,
 (1, 1): 1,
 (1, 2): 1,
 (1, 3): 1,
 (2, 0): 1,
 (2, 1): 1,
 (2, 2): 1,
 (2, 3): 1,
 (3, 0): 1,
 (3, 1): 1,
 (3, 2): 1,
 (3, 3): 1}

In [15]:
### variables
# add x_ijm
x = {}
# x = []
for i in range(I):
    # x.append([])
    for j in range(J):
        # x[i].append([])
        for m in range(M):
            # x[i][j].append(peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "x"+ str(i+1) + str(j+1) + str(m+2)));
             x[i,j,m] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "x"+ str(i+1) + str(j+1) + str(m+2))

# add w_j
w = {}
for j in range(J):
    w[j] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "w"+str(j+1))

# add z_iji'j'
z = {}
for i in range(I):
    # z.append([])
    for j in range(J):
        # z[i].append([])
        for i_ in range(I):
            # z[i][j].append([])
            for j_ in range(J):
                z[i,j,i_,j_] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(i+1) + str(j+1) + str(i_+1) + str(j_+1))

# add y_iji'j'm
y = {}
for i in range(I):
    # y.append([])
    for j in range(J):
        # y[i].append([])
        for i_ in range(I):
            # y[i][j].append([])
            for j_ in range(J):
                # y[i][j][i_].append([])
                for m in range(M):
                    y[i,j,i_,j_,m] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "y" + str(i+1) + str(j+1) + str(i_+1) + str(j_+1) + str(m+2))

# add v_ij
v = {}
for i in range(I):
    # v.append([])
    for j in range(J):
        v[i, j] = peg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "v" + str(i+1) + str(j+1))


In [16]:
### obj
peg1.setObjective(quicksum(w[j] for j in range(J)), GRB.MINIMIZE)
# 1
peg1.addConstrs((((((v[i, j] + T[i,j] - v[i_, j] <= K * z[i, j,i_,j]) for i in range(I-1)) for j in range(J-1)) for i_ in range(i+1, I)) for j_ in range(j+1, J)), "same_job1")

peg1.addConstrs((((((v[i_,j] + T[i,j] - v[i,j] <= K * z[i,j,i_,j]) for i in range(I-1)) for j in range(J-1)) for i_ in range(i+1, I)) for j_ in range(j+1, J)),"same_job2")



{}

In [17]:
# add vx_ijm
vx = {}
for i in range(I):
    for j in range(J):
        for m in range(M):
            vx[i,j,m] = peg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "vx" + str(i+1) + str(j+1)+str(m+2))



In [18]:
peg1.addConstrs((vx[i,j,m] == v[i,j] * x[i,j,m] for i in range(I) for j in range(J) for m in range(M)), "vx");
peg1.addConstrs((y[i,j,i_,j_,m] == x[i,j,m]*x[i_,j_,m] 
                for i in range(I-1) 
                for j in range(J-1) 
                for i_ in range(i+1,I)
                for j_ in range(j+1,J) 
                for m in range(M)),
                "y_constraint")

{(0, 0, 1, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 2, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 2, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 2, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 2, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 3, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 3, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 3, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 3, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 4, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 4, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 4, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 4, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 5, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 5, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 5, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 5, 3): <gurobi.QConstr Not Yet Added>,


In [19]:
# 1-a
peg1.addConstrs(((v[i,j] + T[i,j] - v[i_,j]) <= K * z[i,j,i_,j] 
    for i in range(I-1) 
    for j in range(J)
    for i_ in range(i+1, I)),"same_job1")

# 1-a
peg1.addConstrs(((v[i_,j] + T[i_,j] - v[i,j]) <= K * (1 - z[i,j,i_,j]) 
    for i in range(I-1) 
    for j in range(J)
    for i_ in range(i+1, I)),"samejob2")

{(0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11, 1): <gurobi.Constr *Awaiting Model Update*>}

In [20]:
# 1-b
peg1.addConstrs(((y[i,j,i_,j_,m] * ((vx[i,j,m] + T[i,j]) - vx[i_,j_,m])) <= (K * z[i,j,i_,j_])
                for i in range(I-1)  
                for i_ in range(i+1, I)  
                for j in range(J-1)  
                for j_ in range(j+1, J)  
                for m in range(M)),"same_machine1")

{(0, 1, 0, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 2, 0): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 2, 1): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 2, 2): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 2, 3): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 3, 0): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 3, 1): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 3, 2): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 3, 3): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 4, 0): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 4, 1): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 4, 2): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 4, 3): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 5, 0): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 5, 1): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 5, 2): <gurobi.QConstr Not Yet Added>,
 (0, 1, 0, 5, 3): <gurobi.QConstr Not Yet Added>,


In [21]:
# 1-b
peg1.addConstrs((y[i,j,i_,j_,m] * (vx[i_,j_,m] + T[i_,j_] - vx[i,j,m]) <= K * (1 - z[i,j,i_,j_]) 
    for i in range(I-1) 
    for j in range(J-1)
    for i_ in range(i+1, I)
    for j_ in range(j+1, J)
    for m in range(M)),"same_machine2")

{(0, 0, 1, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 2, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 2, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 2, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 2, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 3, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 3, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 3, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 3, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 4, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 4, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 4, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 4, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 5, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 5, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 5, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 5, 3): <gurobi.QConstr Not Yet Added>,


In [22]:
import math

In [23]:
# peg1.addConstrs(((x[i,j,m] == 0) >> y[i,j,i_,j_,m] == 0 for i in range(I-1) 
#     for j in range(J-1)
#     for i_ in range(i+1, I)
#     for j_ in range(j+1, J)
#     for m in range(M)), name='y_constraint1')

# peg1.addConstrs(((x[i,j_,m] == 0) >>  y[i,j,i_,j_,m] == 0 for i in range(I-1) 
#     for j in range(J-1)
#     for i_ in range(i+1, I)
#     for j_ in range(j+1, J)
#     for m in range(M)), name='y_constraint2')


In [24]:
# 2
peg1.addConstrs((quicksum(x[i,j,m] for m in range(M)) == 1 for i in range(I) for j in range(J)), "each process is completed by exactly one machine")


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>

In [25]:
# 3
peg1.addConstrs((x[i,j,m] * A[i,j,p] <= C[m,p] for i in range(I) for j in range(J) for p in range(P) for m in range(M)), "whether a machine can do to process type")


{(0, 0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 1): <gurobi.Constr *Awaiting Model Up

In [26]:
# 4
# peg1.addConstrs((v[i+1,j] - v[i,j] == T[i+1,j] for i in range(S[j]-1) for j in range(J)), "split1-1")

# peg1.addConstrs((v[i+1,j] - v[i,j] == T[i+1,j] for i in range(S[j], I-1) for j in range(J)), "split1-2")

# peg1.addConstrs((x[i+1,j,m] == x[i,j,m] for i in range(S[j]-1) for j in range(J) for m in range(M)), "split2-1")

# peg1.addConstrs((x[i+1,j,m] == x[i,j,m] for i in range(S[j], I-1) for j in range(J) for m in range(M)), "split2-2")    


In [27]:
# (datetime.combine(date.today(), D[0]) - datetime.combine(date.today(), b)).total_seconds()

In [28]:
# 5
peg1.addConstrs(
    (v[I-1,j] - ((datetime.combine(date.today(), D[j]) - datetime.combine(date.today(), b)).total_seconds()/3600) <= K*w[j] 
    for j in range(J)),
    "minimum tardiness")


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>}

In [29]:
peg1.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 444 rows, 3108 columns and 288 nonzeros
Model fingerprint: 0xb2ce31eb
Model has 888 quadratic constraints
Variable types: 120 continuous, 2988 integer (2988 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [5e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+06]
  QRHS range       [1e+06, 1e+06]
Presolve removed 384 rows and 2538 columns
Presolve time: 0.01s
Presolved: 3612 rows, 1458 columns, 9176 nonzeros
Variable types: 744 continuous, 714 integer (714 binary)
Found heuristic solution: objective 0.0000000

Explored 1 nodes (0 simplex iterations) in 0.18 seconds (0.03 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective

In [30]:
results = peg1.getVars()
for i in range(int(len(results)/I)):
    s = ""
    for j in range(I):
        ind = i*I + j
        s = s + results[ind].varName + "=" + str(results[ind].X) + " "
    print(s)
print("objective value =", peg1.objVal)
# peg1.computeIIS()
# peg1.write("model.ilp")

x112=-0.0 x113=-0.0 
x114=-0.0 x115=1.0 
x122=1.0 x123=-0.0 
x124=-0.0 x125=-0.0 
x132=-0.0 x133=-0.0 
x134=-0.0 x135=1.0 
x142=-0.0 x143=-0.0 
x144=-0.0 x145=1.0 
x152=1.0 x153=-0.0 
x154=-0.0 x155=-0.0 
x162=-0.0 x163=-0.0 
x164=-0.0 x165=1.0 
x172=-0.0 x173=-0.0 
x174=-0.0 x175=1.0 
x182=-0.0 x183=-0.0 
x184=-0.0 x185=1.0 
x192=1.0 x193=-0.0 
x194=-0.0 x195=-0.0 
x1102=-0.0 x1103=-0.0 
x1104=1.0 x1105=-0.0 
x1112=-0.0 x1113=1.0 
x1114=-0.0 x1115=-0.0 
x1122=1.0 x1123=-0.0 
x1124=-0.0 x1125=-0.0 
x212=1.0 x213=-0.0 
x214=-0.0 x215=-0.0 
x222=1.0 x223=-0.0 
x224=-0.0 x225=-0.0 
x232=1.0 x233=-0.0 
x234=-0.0 x235=-0.0 
x242=1.0 x243=-0.0 
x244=-0.0 x245=-0.0 
x252=-0.0 x253=-0.0 
x254=-0.0 x255=1.0 
x262=-0.0 x263=-0.0 
x264=-0.0 x265=1.0 
x272=-0.0 x273=1.0 
x274=-0.0 x275=-0.0 
x282=1.0 x283=-0.0 
x284=-0.0 x285=-0.0 
x292=1.0 x293=-0.0 
x294=-0.0 x295=-0.0 
x2102=-0.0 x2103=1.0 
x2104=-0.0 x2105=-0.0 
x2112=1.0 x2113=-0.0 
x2114=-0.0 x2115=-0.0 
x2122=1.0 x2123=-0.0 
x2124=-0.0 x212

y112232=0.0 y112233=0.0 
y112234=0.0 y112235=0.0 
y112242=0.0 y112243=0.0 
y112244=0.0 y112245=0.0 
y112252=0.0 y112253=0.0 
y112254=0.0 y112255=0.0 
y112262=0.0 y112263=0.0 
y112264=0.0 y112265=0.0 
y112272=0.0 y112273=0.0 
y112274=0.0 y112275=0.0 
y112282=0.0 y112283=0.0 
y112284=0.0 y112285=0.0 
y112292=0.0 y112293=0.0 
y112294=0.0 y112295=0.0 
y1122102=0.0 y1122103=0.0 
y1122104=0.0 y1122105=0.0 
y1122112=0.0 y1122113=0.0 
y1122114=0.0 y1122115=0.0 
y1122122=0.0 y1122123=0.0 
y1122124=0.0 y1122125=0.0 
y21112=0.0 y21113=0.0 
y21114=0.0 y21115=0.0 
y21122=0.0 y21123=0.0 
y21124=0.0 y21125=0.0 
y21132=0.0 y21133=0.0 
y21134=0.0 y21135=0.0 
y21142=0.0 y21143=0.0 
y21144=0.0 y21145=0.0 
y21152=0.0 y21153=0.0 
y21154=0.0 y21155=0.0 
y21162=0.0 y21163=0.0 
y21164=0.0 y21165=0.0 
y21172=0.0 y21173=0.0 
y21174=0.0 y21175=0.0 
y21182=0.0 y21183=0.0 
y21184=0.0 y21185=0.0 
y21192=0.0 y21193=0.0 
y21194=0.0 y21195=0.0 
y211102=0.0 y211103=0.0 
y211104=0.0 y211105=0.0 
y211112=0.0 y211113=0.0 

vx1102=0.0 vx1103=0.0 
vx1104=0.0 vx1105=0.0 
vx1112=0.0 vx1113=0.0 
vx1114=0.0 vx1115=0.0 
vx1122=0.0 vx1123=0.0 
vx1124=0.0 vx1125=0.0 
vx212=2.699999996693805 vx213=0.0 
vx214=0.0 vx215=0.0 
vx222=1.6 vx223=0.0 
vx224=0.0 vx225=0.0 
vx232=1.6 vx233=0.0 
vx234=0.0 vx235=0.0 
vx242=1.6 vx243=0.0 
vx244=0.0 vx245=0.0 
vx252=0.0 vx253=0.0 
vx254=0.0 vx255=2.6999999980907887 
vx262=0.0 vx263=0.0 
vx264=0.0 vx265=2.6999999980907887 
vx272=0.0 vx273=1.4 
vx274=0.0 vx275=0.0 
vx282=1.6 vx283=0.0 
vx284=0.0 vx285=0.0 
vx292=1.6 vx293=0.0 
vx294=0.0 vx295=0.0 
vx2102=0.0 vx2103=1.0 
vx2104=0.0 vx2105=0.0 
vx2112=2.9999999983701855 vx2113=0.0 
vx2114=0.0 vx2115=0.0 
vx2122=2.0 vx2123=0.0 
vx2124=0.0 vx2125=0.0 
objective value = 0.0


In [31]:
peg1.computeIIS()
peg1.write("model.ilp")

IIS runtime: 0.19 seconds (0.03 work units)


GurobiError: Cannot compute IIS on a feasible model

# Problem 2

# Problem 3

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d46e1ceb-56b3-4729-a7f8-5832e58e15c6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>